In [14]:
import sys
import numpy as np
import math
import random
import json
import requests

import gym
import gym_maze
from gym_maze.envs.maze_manager import MazeManager
from riddle_solvers import *

In [15]:
import numpy as np

import gym
from gym import error, spaces, utils
from gym.utils import seeding
from gym_maze.envs.maze_view_2d import MazeView2D


class MazeEnv(gym.Env):
    metadata = {
        "render.modes": ["human", "rgb_array"],
    }

    ACTION = ["N", "S", "E", "W"]

    def __init__(self, maze_file=None, maze_cells=None, maze_size=None, mode=None, enable_render=True, rescue_item_locations=None, has_loops=False):
        super().__init__()

        self.viewer = None
        self.enable_render = enable_render
        self.rescue_item_locations= rescue_item_locations


        if hasattr(maze_cells, 'shape'):
            self.maze_view = MazeView2D(maze_name="OpenAI Gym - Maze (10 x 10)",
                                        maze_cells=maze_cells,
                                        screen_size=(640, 640), 
                                        enable_render=enable_render, rescue_item_locations=self.rescue_item_locations)
        
        elif maze_file:
            self.maze_view = MazeView2D(maze_name="OpenAI Gym - Maze (%s)" % maze_file,
                                        maze_file_path=maze_file,
                                        screen_size=(640, 640), 
                                        enable_render=enable_render, rescue_item_locations=self.rescue_item_locations)
        elif maze_size:
            self.maze_view = MazeView2D(maze_name="OpenAI Gym - Maze (%d x %d)" % maze_size,
                                        maze_size=maze_size, screen_size=(640, 640),
                                        has_loops=has_loops,
                                        enable_render=enable_render, rescue_item_locations=self.rescue_item_locations)
        else:
            raise AttributeError("One must supply either a maze_file path (str) or the maze_size (tuple of length 2)")

        self.maze_size = self.maze_view.maze_size

        # forward or backward in each dimension
        self.action_space = spaces.Discrete(2*len(self.maze_size))
        self.final_pos = (maze_size[0]-1,maze_size[1]-1)
        # observation is the x, y coordinate of the grid
        low = np.zeros(len(self.maze_size), dtype=int)
        high =  np.array(self.maze_size, dtype=int) - np.ones(len(self.maze_size), dtype=int)
        self.observation_space = spaces.Box(0, 50, dtype=int, shape = (14,))

        # initial condition
        self.state = None
        self.steps_beyond_done = None
        
        # Simulation related variables.
        self.reset()

        # Just need to initialize the relevant attributes
        self.configure()
        self.seed()

        self.target_locations = self.maze_view.rescue_item_locations
        self.target_reached = 0
        self.tot_reward = 0

        # self.maze_view.maze.save_maze('./hacktrick.npy')

        self.rescued = 0
        self.last_pos_x =self.maze_view.robot[0]
        self.last_pos_y = self.maze_view.robot[1]
        
        self.cur_pos_x = 0
        self.cur_pos_y = 0
        
        self.last_action = 0
        self.steps = None
        self.steps_count = 0
        
        self.temp = []
        self.visited = []
    def __del__(self):
        if self.enable_render is True:
            self.maze_view.quit_game()

    def configure(self, display=None):
        self.display = display

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        
        info = {}
        self.last_pos_x = self.maze_view.robot[0]
        self.last_pos_y = self.maze_view.robot[1]
        
        
        if isinstance(action, int):
            self.maze_view.move_robot(self.ACTION[action])
        else:
            self.maze_view.move_robot(self.ACTION[action])

        distances = self.maze_view.get_rescue_items_locations()[0]
        directions = self.maze_view.get_rescue_items_locations()[1]

        info['rescued_items'] = self.maze_view.rescued_items


        self.state = [self.maze_view.robot, distances, directions]
        # self.state = self.maze_view.robot
        # print("new state",self.state_new)
        self.state = self.fix_state()
        
        np.append(self.state, self.last_action)
        self.last_action = action
        
        
        self.cur_pos_x = self.state[0]
        self.cur_pos_y = self.state[1]
        
        truncated = False
        
        if self.steps_count >= 5001:
            truncated = True
            
        info['riddle_type'] = None
        info['riddle_question'] = None
        
        reward = self.compute_reward()
        terminated = False
        self.tot_reward += reward

        if self.tot_reward < -500:
            terminated = True
            reward = -10
            
        self.steps +=1
        self.steps_count+=1
        self.normalize_state()
        
        self.temp = []
        self.temp.append(self.state[0])
        self.temp.append(self.state[1])
        

        return self.state, reward, terminated, info

    def compute_reward(self):
        
        if self.temp not in self.visited:
            self.visited.append(self.temp)
            return 0.1
        reached = False
        
        if self.steps_count > 5000:
            self.terminated = True
            return -1
        
        new_targets = []
        for i in self.target_locations:
            if (self.cur_pos_x == i[0]) and (self.cur_pos_y == i[1]):
                reached = True
                continue
            new_targets.append(i)
        self.target_locations = new_targets
        
        if reached:
            return 0.25
    
        elif (self.state[0]  == self.last_pos_x) and (self.last_pos_y == self.state[1]):
            return -0.1
        
        elif (self.state[0] == 9) and (self.state[1] == 9) :
            self.terminated = True
            return 0.25 * (4 - len(self.target_locations))
        
        else:
            return -0.01
        
        
        
    def get_current_state(self):
        info = {}
        distances = self.maze_view.get_rescue_items_locations()[0]
        directions = self.maze_view.get_rescue_items_locations()[1]

        info['rescued_items'] = self.maze_view.rescued_items


        self.state = [self.maze_view.robot, distances, directions]
        truncated = False
        
        info['riddle_type'] = None
        info['riddle_question'] = None
        
        reward = None
        terminated = False

        return self.state, reward, terminated, truncated, info

    def reset(self):
        self.rescue_item_locations = self.get_rescue_points()
        self.seed()
        self.target_locations = self.maze_view.rescue_item_locations
        self.target_reached = 0
        self.tot_reward = 0
        # self.maze_view.maze.save_maze('./hacktrick.npy')
        self.rescued = 0
        self.last_pos_x =self.maze_view.robot[0]
        self.last_pos_y = self.maze_view.robot[1]
        self.cur_pos_x = 0
        self.cur_pos_y = 0
        self.last_action = 0
        self.steps = None
        self.steps_count = 0
        self.visited = []
        self.maze_view.reset_robot()
        self.state = [self.maze_view.robot, self.maze_view.get_rescue_items_locations()[0], self.maze_view.get_rescue_items_locations()[1]]
        self.steps_beyond_done = None
        self.steps = 0
        self.terminated = False
        self.truncated = False
        self.maze_view.reset_rescue_items()
        self.state = self.fix_state()
        self.tot_reward = 0
        self.temp = []
        self.visited = []
        self.last_pos_x = 0
        self.last_pos_y = 0
        self.last_action =0
        np.append(self.state, self.last_action)
        self.pos = (0,0)
        self.target_locations = self.maze_view.rescue_item_locations
        self.normalize_state()
        return self.state

    def is_game_over(self):
        return self.maze_view.game_over

    def render(self, mode="human", close=False):
        if close:
            self.maze_view.quit_game()

        return self.maze_view.update(mode)

    def fix_state(self):
        state = self.state
        state0 = state[0]
        state1 = state[1]
        state2 = state[2]
    
        ans = []
        for i in state0:
            ans.append(i)
        for i in state1:
            ans.append(i)
        for i in state2:
            for j in i:
                ans.append(j)
        return np.array(ans, dtype=float)
    
    def normalize_state(self):
        dis_sum = sum(self.state[2:6]) 
        self.state[2] /= dis_sum
        self.state[3] /= dis_sum
        self.state[4] /= dis_sum
        self.state[5] /= dis_sum
        
        self.state[0] /= 9
        self.state[1] /= 9
    
    def get_rescue_points(self):
        random_tuples = set()  # Create an empty set to hold the tuples 

        while len(random_tuples) < 4:  # Continue until we have 4 unique tuples 
            x = random.randint(1, 8) 
            y = random.randint(1, 8) 
            new_tuple = (x, y) 
            if new_tuple not in random_tuples:  # Only add the tuple if it's not already in the set 
                random_tuples.add(new_tuple) 
        return random_tuples

        
class MazeEnvSample5x5(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvSample5x5, self).__init__(maze_file="maze2d_5x5.npy", enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)


class MazeEnvRandom5x5(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvRandom5x5, self).__init__(maze_size=(5, 5), enable_render=enable_render, maze_cells=maze_cells, mode='plus', rescue_item_locations= rescue_item_locations)


class MazeEnvSample10x10(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None, maze_file=None):
        super(MazeEnvSample10x10, self).__init__(maze_file=maze_file, enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)


class MazeEnvRandom10x10(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvRandom10x10, self).__init__(maze_size=(10, 10), enable_render=enable_render, maze_cells=maze_cells, mode='plus', rescue_item_locations=rescue_item_locations)


class MazeEnvSample3x3(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvSample3x3, self).__init__(maze_file="maze2d_3x3.npy", enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)


class MazeEnvRandom3x3(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvRandom3x3, self).__init__(maze_size=(3, 3), enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)


class MazeEnvSample100x100(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvSample100x100, self).__init__(maze_file="maze2d_100x100.npy", enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)


class MazeEnvRandom100x100(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvRandom100x100, self).__init__(maze_size=(100, 100), enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)


class MazeEnvRandom10x10Plus(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvRandom10x10Plus, self).__init__(maze_size=(10, 10), mode="plus", enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)


class MazeEnvRandom20x20Plus(MazeEnv):

    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvRandom20x20Plus, self).__init__(maze_size=(20, 20), mode="plus", enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)


class MazeEnvRandom30x30Plus(MazeEnv):
    def __init__(self, enable_render=True, maze_cells=None, rescue_item_locations=None):
        super(MazeEnvRandom30x30Plus, self).__init__(maze_size=(30, 30), mode="plus", enable_render=enable_render, maze_cells=maze_cells, rescue_item_locations= rescue_item_locations)

In [16]:
rescue_items_location = [(1, 1), (8, 8), (4,8),(3,5)]

In [17]:
sample_maze = np.load("hackathon_sample.npy")

In [18]:
env = MazeEnv(maze_file='hackathon_sample.npy',maze_size=(10,10), rescue_item_locations=rescue_items_location, mode='human', maze_cells=sample_maze)

In [19]:
import cv2

In [20]:
import keyboard
env.reset()
action=0
ss = ["N", "S", "E", "W"]

while True:
    cv2.waitKey(0)
    if keyboard.is_pressed("q"):
        break
    elif keyboard.is_pressed("w"):
        action = 0
    elif keyboard.is_pressed('s'):
        action=1
    elif keyboard.is_pressed('a'):
        action=3
    elif keyboard.is_pressed('d'):
        action = 2
    state,reward,done,info = env.step(action)
    img = env.render('rgb_array')
    img = cv2.resize(img,(720,720))
    img = cv2.putText(img,org=(50,20),text=f"reward:{reward:.3f},{done}",
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.5,color=(0,255,0))
    cv2.imshow('sss',img)
    
cv2.destroyAllWindows()